## Import des librairies ##

In [5]:
#Imports librairies de base
import numpy as np
import pandas as pd

#Affichage de Graphes
from IPython.display import Markdown, display, HTML
import matplotlib.pyplot as plt

#Information Mutuelle
from sklearn.feature_selection import mutual_info_regression


## 1. Lecture des données ##

:1.1 Charger les données d'apprentissage (Data/passagers.csv) dans un DataFrame nommé train. Afficher 10 exemples.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train = pd.read_csv('/content/drive/MyDrive/Data_Science/passagers.csv')
print(train.head(10))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   
6            7         0       1   
7            8         0       3   
8            9         1       3   
9           10         1       2   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   
5                                   Moran, Mr. James    male   NaN      0   
6                            McCarthy, Mr. Timothy J    male  54

1.2 Charger les données d'apprentissage (Data/test.csv) dans un Dataframe nommé test. Afficher 10 exemples.

In [8]:
test = pd.read_csv('/content/drive/MyDrive/Data_Science/test.csv')
print(test.head(10))

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   
5          897       3                    Svensson, Mr. Johan Cervin    male   
6          898       3                          Connolly, Miss. Kate  female   
7          899       2                  Caldwell, Mr. Albert Francis    male   
8          900       3     Abrahim, Mrs. Joseph (Sophie Halaut Easu)  female   
9          901       3                       Davies, Mr. John Samuel    male   

    Age  SibSp  Parch     Ticket     Fare Cabin Embarked  
0  34.5      0      0     330911   7.8292   NaN        Q  
1

1.3 Afficher les informations des deux DataFrames train et Test.

In [ ]:
# les infos du DataFrame train
print("Informations du DataFrame train :")
print(train.info())
print("\nPremières lignes du DataFrame train :")
print(train.head())

# les infos du DataFrame test
print("\nInformations du DataFrame test :")
print(test.info())
print("\nPremières lignes du DataFrame test :")
print(test.head())

Informations du DataFrame train :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

Premières lignes du DataFrame train :
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4        

1.4 Calculer le taux global de survie

In [ ]:
# Survival DataFrame train
survival_rate = train['Survived'].mean()
print(f"Taux global de survie : {survival_rate:.2%}")

Taux global de survie : 38.38%


#### Question 1 : Quel Travail voyez-vous pour que les données soient prêtes à faire tourner dans un modèle ?
Pour préparer les données à être utilisées dans un modèle, des étapes de nettoyage et de transformation sont nécessaires. Par exemple, une étape de gestion de valeur manquante, la création de nouveaux targets, de l'encodage.




#### Question 2 : Quelle différence y-a-t'il entre les données train et les données test ?

Le DataFrame train contient un champ supplémentaire (Survived), qui indique si un passager a survécu. "1" si oui, "0" si non.




#### Question 3 : Quel est le rôle de ces deux sets de données ?
Le dataframe Train a pour role de jeu d'entraînement pour former le modèle. Ce dataframe nous apprend les relations entre les features et la variable cible

Le dataframe Test a pour role d'évaluer le modèle sur des données qu'il n'a jamais vues.



## 2. Analyse des features

2.1 Analyser l'effet de la variable Classe sur le taux de survie, en affichant le % de survie par classe

In [11]:
# Analyse de la variable Classe (Pclass) sur le taux de survie
class_survival = train.groupby('Pclass')['Survived'].mean() * 100
print("Taux de survie par classe :")
print(class_survival)

Taux de survie par classe :
Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64


2.2 Analyser l'effet de la variable Genre (Sex) sur le taux de survie, en affichant le % de survie par classe

In [12]:
# Analyse la variable Genre (Sex) sur le taux de survie
gender_survival = train.groupby('Sex')['Survived'].mean() * 100
print("Taux de survie par genre :")
print(gender_survival)

Taux de survie par genre :
Sex
female    74.203822
male      18.890815
Name: Survived, dtype: float64


2.3 Calculer une nouvelle colonne FamilySize contenant la taille de la famille, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et la taille de la famille.

In [13]:
# Ajout de la colonne FamilySize pour "train" et "test"
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

# DataFrame train
print("Aperçu de la colonne FamilySize dans le DataFrame train :")
print(train[['PassengerId', 'Name', 'Age', 'Sex', 'FamilySize']].head(10))

# DataFrame test
print("\nAperçu de la colonne FamilySize dans le DataFrame test :")
print(test[['PassengerId', 'Name', 'Age', 'Sex', 'FamilySize']].head(10))


Aperçu de la colonne FamilySize dans le DataFrame train :
   PassengerId                                               Name   Age  \
0            1                            Braund, Mr. Owen Harris  22.0   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
2            3                             Heikkinen, Miss. Laina  26.0   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
4            5                           Allen, Mr. William Henry  35.0   
5            6                                   Moran, Mr. James   NaN   
6            7                            McCarthy, Mr. Timothy J  54.0   
7            8                     Palsson, Master. Gosta Leonard   2.0   
8            9  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  27.0   
9           10                Nasser, Mrs. Nicholas (Adele Achem)  14.0   

      Sex  FamilySize  
0    male           2  
1  female           2  
2  female           1  
3  female           

2.4 Visualisez le nombre de personnes et le taux de survie par taille de famille.

In [14]:
# Visualisation du nombre de personnes et le taux de survie par taille de famille
family_survival = train.groupby('FamilySize')['Survived'].agg(['count', 'mean'])
family_survival['mean'] = family_survival['mean'] * 100

print("\nNombre de personnes et taux de survie par taille de famille :")
print(family_survival)


Nombre de personnes et taux de survie par taille de famille :
            count       mean
FamilySize                  
1             537  30.353818
2             161  55.279503
3             102  57.843137
4              29  72.413793
5              15  20.000000
6              22  13.636364
7              12  33.333333
8               6   0.000000
11              7   0.000000


2.5 Calculer une nouvelle colonne IsAlone indiquant si le passager est seul ou pas, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et le status IsAlone.

In [15]:
train['IsAlone'] = (train['FamilySize'] == 1).astype(int)
test['IsAlone'] = (test['FamilySize'] == 1).astype(int)

print("\nExemples de la colonne IsAlone (train) :")
print(train[['PassengerId', 'Name', 'Age', 'Sex', 'IsAlone']].head(10))

print("\nExemples de la colonne IsAlone (test) :")
print(test[['PassengerId', 'Name', 'Age', 'Sex', 'IsAlone']].head(10))


Exemples de la colonne IsAlone (train) :
   PassengerId                                               Name   Age  \
0            1                            Braund, Mr. Owen Harris  22.0   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
2            3                             Heikkinen, Miss. Laina  26.0   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
4            5                           Allen, Mr. William Henry  35.0   
5            6                                   Moran, Mr. James   NaN   
6            7                            McCarthy, Mr. Timothy J  54.0   
7            8                     Palsson, Master. Gosta Leonard   2.0   
8            9  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  27.0   
9           10                Nasser, Mrs. Nicholas (Adele Achem)  14.0   

      Sex  IsAlone  
0    male        0  
1  female        0  
2  female        1  
3  female        0  
4    male        1  
5    m

2.6 Visualisez le nombre de personnes et le taux de survie par statut seul ou pas.

In [16]:
# Visualisation par statut seul ou pas
isalone_survival = train.groupby('IsAlone')['Survived'].mean()

print("Taux de survie par statut seul ou pas")
print(isalone_survival)


Taux de survie par statut seul ou pas
IsAlone
0    0.505650
1    0.303538
Name: Survived, dtype: float64


2.7 Calculer l'information mutuelle sur les données train.

Attention : Il y a des données continues et des données discrètes

Ignorer les colonnes Name, Ticket et Cabin

2.8 Afficher dans un graphique les scores d'information mutuelle par feature

### Question 1 : La variable Classe est-elle utile pour prévoir la survie ? Pourquoi ?



### Question 2 : Même question pour la variable Sex


### Question 3 : Comment traiter le variables liées à la famille du passager ?





## 3. Traitement des données manquantes (à faire sur les données Train et Test)

3.1 La variable "Embarked" possède des valeurs nulles, compléter par la valeur majoritaire (S).

3.2 Afficher l'influence du port d'embarquement sur la Survie

3.3 La variable "Fare" (prix du ticket) possède des valeurs manquantes. Remplacer ces valeurs par la valeur médiane.

3.4 Créer une variable CategoricalFare Découpant la valeur de ticket en 4 catégories de tailles à peu près égales.

3.5 Analyser l'effet de ces catégories de prix de ticket sur la survie.

3.6 Combien de valeurs d'Age manquent ? Complète par des valeurs entières aléatoires entre (moyenne - écart type) et (moyenne + écart type).

3.7 Créer une variable CategoricalAge Découpant la valeur d'age en 5 tranches d'age de même taille.

3.8 Analyser l'effet de ces catégories de prix de ticket sur la survie.

### Question 1 : Classer les variables par ordre d'intérêt

## 4 : Traitement de features complexes

4.1 : Le nom permet de retrouver le titre du passager. Trouver les titres lorsqu'ils existent.

4.2 : En utilisant la fonction pandas crosstab, afficher pour train le nombre d'occurences par titres (en ligne) et par sexe (en colonne)

4.3 :
Remplacer 'Melle et 'Ms' par 'Miss'

Remplacer 'Mme' par 'Mrs'

Creer une valeur "Rare" pour les autres titres avec moins de 10 occurences.

Calculer ensuite le taux de survie par titre

## 5. Mise en forme des données non numériques

5.1 Attribuer une valeur numérique au Genre. Par exemple 0 pour 'female' et 1 pour 'male'

5.2 : Attribuer une valeur pour les possibilités de la variable "Title" ('Master','Miss','Mr','Mrs','Rare')

5.3 Remplacer les titres manquant par 0, et attribuer à la colonne le type entier

5.4 Attribuer une valeur numérique aux 4 catégories de la variable Fare

5.5 Attibuer une valeur numérique au Port d'embarquement (Embarked)

5.6 Supprimer les features inutiles

5.7 Afficher les infos pour les deux datasets

5.8 Sauver les deux datasets dans des fichier csv. Eviter d'écrire un index (n° de ligne)